# Библиотеки

In [1]:
import random
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import scipy.stats as ss
from typing import Union, List, Dict, Tuple
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

# Конфиг

In [2]:
warnings.filterwarnings('ignore')

with open('config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
path_to_data = config['path_to_data']
seed = config['random_state']

%config InlineBackend.figure_formats = config['backend_figure_formats']
random.seed(seed)
np.random.seed(seed)
# pd.set_option('display.precision', config['display_precision'])

# Разведочный анализ данных

In [3]:
transactions = pd.read_parquet(f'{path_to_data}transactions.parquet')
print('Число транзакций:', len(transactions))

# Беру sample, а не head/tail, чтобы порядок данных не исказил представление о них
transactions.sample(5, random_state=seed)

Число транзакций: 7620119


,sku_id,price,number,cheque_id,litrs,client_id,shop_id,date
2466387,1705,0.008979,0.000452,1037320,0.104183,152741.0,63,2171-06-12
1930834,1163,0.002509,0.000000,2334351,0.144883,NaN,40,2171-06-23
7440318,1140,0.006567,0.000452,3290322,0.104183,NaN,102,2171-03-04
2446771,1159,0.002586,0.000000,877192,0.123961,9156.0,18,2171-06-12
673570,2226,0.009184,0.000452,3007243,0.104183,123659.0,73,2171-07-17


In [4]:
# Посмотрим, в порядке ли всё с типами колонок
# Хотелось бы client_id заменить на int (вряд ли айди - флоат), чтобы не тратить память;
# Но просто это не сделать - есть пропуски (см. дальше), оставлю пока что так
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7620119 entries, 0 to 7620118
Data columns (total 8 columns):
 #   Column     Dtype         
---  ------     -----         
 0   sku_id     int64         
 1   price      float64       
 2   number     float64       
 3   cheque_id  int64         
 4   litrs      float64       
 5   client_id  float64       
 6   shop_id    int64         
 7   date       datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 465.1 MB


In [5]:
# Отложим кусок выборки, чтобы не подогнать данные под свои гипотезы
# (заодно, он выступит тестом для проверки моделей)
transactions.sort_values(by=['date', 'cheque_id'], inplace=True)
train_transactions, test_transactions = train_test_split(transactions, test_size=0.3, shuffle=False)

del transactions  # чтобы не тратить память

In [6]:
sku_info = pd.read_parquet(f'{path_to_data}nomenclature.parquet')
print('Число товаров в номенклатуре:', len(sku_info))
sku_info.sample(5, random_state=seed)

Число товаров в номенклатуре: 5103


,sku_id,full_name,brand,sku_group,OTM,units,country
4137,2816,unknown,unknown,Соки и сокосодержащие напитки,unknown,unknown,unknown
586,634,Масло Lubricrol Magnatec 5W-30 A3/B4 1л,Lubricrol,Масла моторные (для Ethereumовых двигателей),Нет,л,ГЕРМАНИЯ
553,609,Масло G-Energy F Synth 5W-40 1л,G-Energy,"Масла моторные (для Ethereumовых двигателей) ""...",Нет,л,ИТАЛИЯ
227,3644,Семечки БАБКИНЫ СЕМЕЧКИ 100г,БАБКИНЫ СЕМЕЧКИ,Снеки,Нет,г,unknown
1321,1869,Ароматизатор Areon FRTN17 Fresco black crystal,Areon,Уход за автомобилем,Нет,мл,БОЛГАРИЯ


In [7]:
sku_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5103 entries, 0 to 5102
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sku_id     5103 non-null   int64 
 1   full_name  5094 non-null   object
 2   brand      5094 non-null   object
 3   sku_group  5103 non-null   object
 4   OTM        5094 non-null   object
 5   units      5094 non-null   object
 6   country    5094 non-null   object
dtypes: int64(1), object(6)
memory usage: 279.2+ KB


In [8]:
# "unknown" заменю на np.nan
sku_info.replace(to_replace='unknown', value=np.nan, inplace=True)

# Приведем названия к нижнему регистру для удобства
for col in sku_info.columns:
    try:
        sku_info[col] = sku_info[col].str.lower()
    except:
        print(col)

sku_id


In [9]:
# В транзакциях пропуски есть только у айди клиента, и их половина
# К тому же, у нас нету фичей клиентов
# => можно посмотреть в сторону item to item подхода
train_transactions.isnull().mean()

sku_id       0.00000
price        0.00000
number       0.00000
cheque_id    0.00000
litrs        0.00000
client_id    0.49086
shop_id      0.00000
date         0.00000
dtype: float64

In [10]:
# В номенклатуре больше половины данных с каким-то пропуском

sku_info.isnull().sum()

sku_id          0
full_name    1379
brand        1733
sku_group       0
OTM          1379
units        1727
country      2195
dtype: int64

In [11]:
none_statement = lambda df: df.isnull().any(axis=1)
none_sku = sku_info.loc[none_statement(sku_info)]
none_sku.sample(15, random_state=seed)

,sku_id,full_name,brand,sku_group,OTM,units,country
4552,4927,NaN,NaN,общественное питание,NaN,NaN,NaN
3504,2265,семена поиск анютины глазки викторианская смес...,NaN,сезонные товары,нет,шт,NaN
3893,2789,NaN,NaN,общественное питание,NaN,NaN,NaN
3943,2897,NaN,NaN,снеки,NaN,NaN,NaN
3530,3693,перчатки palisad 67743 садовые салатовые l,NaN,сезонные товары,нет,шт,китай
5041,5053,NaN,NaN,прочие напитки кафе,NaN,NaN,NaN
3459,4696,коптильня союзгриль n1-f14 одноразовая щепа,союзгриль,сезонные товары,нет,шт,NaN
411,28,леденцы orbit натуральная мята 35г,orbit,кондитерские изделия,нет,г,NaN
4516,341,NaN,NaN,уход за автомобилем,NaN,NaN,NaN
4695,2107,NaN,NaN,соки и сокосодержащие напитки,NaN,NaN,NaN


In [12]:
# 59% транзакций с этими товарами, не хочется терять о них информацию
train_transactions_with_none_sku = train_transactions.loc[train_transactions.sku_id.isin(set(none_sku['sku_id']))]

print(f'Число транзакций с "плохими" sku: {len(train_transactions_with_none_sku)}')
print(f'Доля таких транзакций: {len(train_transactions_with_none_sku) / len(train_transactions):.2f}')

Число транзакций с "плохими" sku: 3156483
Доля таких транзакций: 0.59


Для товара с "топливной" sku_group легко заполнить units - литры, \
Ethereum 92 и пр., скорее всего, тоже топливо - возьмём литры.

In [13]:
# Важно отсеять масла, в группе которых тоже есть слово подстрока "ethereum")
sku_info['sku_group'].value_counts()

кондитерские изделия                                               714
сезонные товары                                                    649
автотовары                                                         516
общественное питание                                               441
уход за автомобилем                                                426
хозяйственные товары, персональный уход                            423
снеки                                                              360
табачные изделия                                                   264
гастроном                                                          196
сладкие уранированные напитки, холодный чай                        184
соки и сокосодержащие напитки                                      145
вода                                                               144
прочие напитки кафе                                                144
бакалея                                                             95
очки д

In [14]:
oil_condition = lambda df: (
    df['sku_group'].str.contains('топливо') |
    df['sku_group'].str.contains('ethereum') &
    df['sku_group'].str.contains('масла')
)

sku_info.loc[oil_condition(sku_info), 'units'] = 'л'

# Если посмотреть на данные, то можно заполнить пропуски еще так:
sku_info.loc[sku_info['full_name'].str.contains(r'[0-9]+\г', na=False), 'units'] = 'г'  # Если в названии есть что-то вроде "18г"
sku_info.loc[sku_info['full_name'].str.contains(r'[0-9]+\л', na=False), 'units'] = 'л'  # Аналогично с л и шт
sku_info.loc[sku_info['full_name'].str.contains(r'[0-9]+\шт', na=False), 'units'] = 'шт'

In [15]:
# Стало получше
sku_info['units'].isnull().sum()

1626

In [16]:
good_sku = sku_info.loc[~none_statement(sku_info)]

In [17]:
# Была надежда, что можно будет найти sku_group, по которой можно однозначно восстановить признак OTM
# Но нет
our_goods = good_sku.loc[good_sku['OTM']=='да']
foreign_goods = good_sku.loc[good_sku['OTM']=='нет']

set(our_goods['sku_group']).issubset(foreign_goods['sku_group'])

True

Самый простой вариант победить пропуски в номенклатуре - заполнить их модой по каждому столбцу.\
Но это очень топорное решение -- есть строки с известными фичами, по таким строкам можно, например, оценить один признак и по нему восстановить другие.\

Но дальше по коду я буду получать векторное представление товаров с помощью one-hot кодирования и учту пропуски как отдельную переменную

In [18]:
# sku_info =  pd.concat([none_sku, good_sku])
# sku_info.fillna(sku_info.mode().iloc[0], inplace=True)

# del good_sku, none_sku

In [19]:
# Литры и штуки все положительные, это хорошо
np.all(train_transactions.litrs >= 0) & np.all(train_transactions.number >= 0)

True

In [20]:
# Преобразуем дату в более информативные признаки

def create_date_features(df: pd.DataFrame, date_column: Union[List[str], str]) -> None:
    # Функция принимает ссылку на датафрейм и меняет его, поэтому ничего не возвращает
    df['year'] = df[date_column].dt.year
    df['month'] = df[date_column].dt.month
    df['week'] = df[date_column].dt.isocalendar().week
    df['day'] = df[date_column].dt.day
    df['dayofweek'] = df[date_column].dt.dayofweek
    df.drop(columns='date', inplace=True)

In [21]:
create_date_features(train_transactions, 'date')
train_transactions.sample(3, random_state=seed)

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,year,month,week,day,dayofweek
5311419,1827,0.002822,0.000452,49491,0.104183,103147.0,18,2171,4,15,14,6
2616330,3861,0.013802,0.000452,1791643,0.104183,NaN,77,2171,6,23,9,6
3996809,3708,0.002822,0.000452,785453,0.104183,82841.0,68,2171,5,19,12,6


In [22]:
# Посмотрим, есть ли записи, относящиеся к одному чеку, но с разными/иногда пропущенными client_id
# (вдруг система дала сбой и не для всех товаров чека записала client_id)

np.all(
    train_transactions
    .fillna(-1)
    .groupby('cheque_id')['client_id']
    .agg(lambda x: len(set(x))) == 1)

# Нет, всё однозначно
# Значит, по чекам, для которых есть client_id, восстановить пропущенные client_id не получится

True

In [23]:
# Есть транзакции с number > 0 и litrs > 0, чего быть не должно
# (товар либо топливо, либо не топливо)

wtf_goods_cond = lambda df: (
    (df['number'] > 0) &
    (df['litrs'] > 0)
)
train_transactions.loc[wtf_goods_cond(train_transactions)].sample(3, random_state=seed)

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,year,month,week,day,dayofweek
3923487,2960,0.008466,0.000452,1999871,0.104183,308314.0,49,2171,5,20,14,1
5580263,2225,0.008671,0.000452,187088,0.104183,NaN,69,2171,4,15,9,1
2328989,119,0.003797,0.000452,3280804,0.104183,NaN,100,2171,6,24,15,5


In [24]:
# Обнулим number у записей с продажей топлива и litrs с продажей не топлива
train_transactions = pd.merge(
    left=train_transactions,
    right=sku_info,
    on='sku_id',
    how='left'
)
train_transactions.loc[oil_condition(train_transactions), 'number'] = 0
train_transactions.loc[~oil_condition(train_transactions), 'litrs'] = 0

In [25]:
train_transactions.sample(5, random_state=seed)

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,year,month,week,day,dayofweek,full_name,brand,sku_group,OTM,units,country
2261766,1827,0.002822,0.000452,49491,0.0,103147.0,18,2171,4,15,14,6,напиток fly cafe (стм)/g-fly безалкогольный не...,fly cafe (стм),вода,да,л,NaN
5017532,3861,0.013802,0.000452,1791643,0.0,NaN,77,2171,6,23,9,6,"леденцы halls mini mints цитрусовый пунш 12,5г",halls,кондитерские изделия,нет,г,турция
3612392,3708,0.002822,0.000452,785453,0.0,82841.0,68,2171,5,19,12,6,вода aqua minerale питьевая уранированная чере...,aqua minerale,вода,нет,л,россия
1709550,964,0.009492,0.000452,2443016,0.0,NaN,54,2171,4,14,2,1,сигареты kiss super slims dream,NaN,табачные изделия,нет,NaN,россия
199706,834,0.007029,0.000452,2404746,0.0,51298.0,78,2171,3,10,4,0,сигареты winston super slims silver пачка,winston,табачные изделия,нет,шт,россия


# Бейзлайн

В качестве бейзлана будем рекомендовать топ-20 самых популярных товаров из категорий, по которым хотят увеличить продажи

In [26]:
desired_groups = {
    'вода',
    'сладкие газированные напитки, холодный чай',
    'кофейные напитки с молоком',
    'энергетические напитки',
    'снеки',
    'соки и сокосодержащие напитки'
}

top_20_products = pd.DataFrame(
    train_transactions
    .loc[train_transactions['sku_group'].isin(desired_groups)]
    .groupby('sku_id')['sku_id']
    .count()
    .sort_values(ascending=False)
    [:20]
).rename(columns={'sku_id': 'count'})

In [27]:
top_20_products.join(sku_info.set_index('sku_id'))

,count,full_name,brand,sku_group,OTM,units,country
sku_id,,,,,,,
3329,99324,напиток fly cafe молокосодержащий кофейный мок...,fly cafe,кофейные напитки с молоком,нет,л,россия
3324,76094,напиток fly cafe молокосодержащий кофейный флэ...,fly cafe,кофейные напитки с молоком,да,л,NaN
3334,45861,"кофе fly cafe латте холодный 0,2л",fly cafe,кофейные напитки с молоком,нет,л,NaN
1551,38332,NaN,NaN,кофейные напитки с молоком,NaN,NaN,NaN
3330,36327,напиток fly cafe молокосодержащий кофейный лат...,fly cafe,кофейные напитки с молоком,да,л,NaN
2673,35840,NaN,NaN,кофейные напитки с молоком,NaN,NaN,NaN
808,33478,напиток g-fly (стм) original энергетический бе...,g-fly (стм),энергетические напитки,нет,л,россия
1535,25169,вода fly cafe (стм)/g-fly питьевая неуранирова...,fly cafe (стм),вода,нет,л,россия
3336,24635,напиток fly cafe молокосодержащий кофейный мок...,fly cafe,кофейные напитки с молоком,нет,л,россия


Посчитаем MAP@20 на тестовой выборке

In [28]:
test_actual = pd.DataFrame(
    test_transactions
    .groupby('cheque_id')['sku_id']
    .apply(list)
    .values.tolist()
).values

In [29]:
# Возникает побочный эффект в виде np.nan, но это не помешает делу
test_actual

array([[1180., 3334., 4587., ...,   nan,   nan,   nan],
       [1158.,  834.,   nan, ...,   nan,   nan,   nan],
       [ 824.,  112., 1344., ...,   nan,   nan,   nan],
       ...,
       [1159., 1159.,   nan, ...,   nan,   nan,   nan],
       [1158., 1158.,   nan, ...,   nan,   nan,   nan],
       [1158., 1158.,  151., ...,   nan,   nan,   nan]])

In [30]:
test_predicted = np.tile(
    top_20_products.index.values,
    (len(test_actual), 1)
)

In [31]:
from metrics import mean_average_precision_at_k


mean_average_precision_at_k(test_actual, test_predicted)  # 0.00364

  0%|          | 0/867282 [00:00<?, ?it/s]

0.0036351249992426483

Метрика не очень высокая, но это простой бейзлан, и важно понимать, что MAP@K **не подходит** для нашей задачи. Мы хотим увеличить продажи **определённых** групп товаров, и далеко не факт, что люди покупали эти товары на истории, поэтому и значение метрики будет небольшим. Если пытаться решить этот вопрос и, например, на истории брать только продажи заданных групп и считать метрику для бейзлайна, то MAP@K будет равен 1, но модель, очевидно, не лучшая.

Точно подошла бы онлайн-метрика: в ритейле таковыми чаще всего выступают маржинальность и розничный товарооборот. Можно было бы провести A/B-тест (в одной группе АЗС продавец рекомендует любой товар из желаемых групп, в другой товар, который наша модель считает наиболее вероятным) и посмотреть, действительно ли наша модель увеличивает продажи (РТО) этих товаров.

Конечно, никто не будет делать A/B тест модели с непонятными оффлайн-метриками и их значениями, поэтому это открытый вопрос.

# Векторные представления для товаров и чеков

Закодируем признаки sku с помощью фиктивных переменных и уменьшим размерность.\
С бинарными признаками хорошо справляется Multiple Correspondence Analysis, но в известных библиотеках он не имплементирован, поэтому использовать будем PCA.

Так мы получим векторные представления товаров. В качестве векторного представления чека возьмём среднее векторных представлений товаров, принадлежащих этому чеку, и для нового чека будет рекомендовать ближайший к нему в евклидовом смысле товар из желаемых категорий.

In [32]:
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors


encoded_sku_info = pd.get_dummies(
    sku_info
    .drop(columns='full_name')
    .set_index('sku_id'),
    dummy_na=True  # считаем пропуск значением переменной
)

# Если посмотреть, то 100 компонент сохраняют 90% дисперсии
# Я оставил примерно 20, чтобы последующие вычисления занимали не так много времени
pca = PCA(n_components=0.8, random_state=seed)
transformed_sku_info = pd.DataFrame(
    pca.fit_transform(encoded_sku_info),
    index=encoded_sku_info.index
)

In [33]:
# Для поиска ближайшего товара будем использовать КД-дерево
from sklearn.neighbors import KDTree

desired_sku = sku_info.loc[sku_info['sku_group'].isin(desired_groups), 'sku_id'].values
tree = KDTree(transformed_sku_info.loc[desired_sku].values)

In [34]:
from metrics import average_precision_at_k


apk = []
for actual in tqdm(test_actual):
    actual = actual[~np.isnan(actual)]
    embedding = transformed_sku_info.loc[actual].sum().values  # эмбеддинг чека
    _, predicted = tree.query(embedding.reshape(1, -1), k=20)
    predicted = predicted[0]
    
    apk.append(average_precision_at_k(actual, predicted))

print(f'MAP@K: {np.mean(apk)}')

  0%|          | 0/867282 [00:00<?, ?it/s]

MAP@K: 0.00010524717679569223


Подход с эмбеддингами на основе one hot encoding - не лучшая идея, и, как и следовало ожидать, метрика не самая высокая.

# User-Item матрица и случайные блуждания

Попробуем подход со случайными блужданиями по матрице смежности графа (привычную матрицу "юзер-айтем", которая часто всплывает в задаче ранжирования/рекомендаций можно мыслить как матрицу смежности двудольного графа), описанный в следующей статье: https://nms.kcl.ac.uk/colin.cooper/papers/recommender-rw.pdf

Проблема подхода в том, что мы не сможем порекомендовать товары новому клиенту (здесь можно посмотреть в сторону Alternating Least Squares, который можно быстро выучивать векторное представление пользователя по уже имеющимся для "юзеров" и "айтемов"). Поэтому здесь я не буду делить выборку транзакций на трейн и тест.

In [35]:
# С матрицей будем работать в разреженном виде
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize

In [36]:
# Модель буду строить и проверять на случайной подвыборке транзакций, иначе мне не хватит памяти

def clear_and_sample_transactions(transactions: pd.DataFrame,
                                  size: int = 10**6) -> pd.DataFrame:
    clear_transactions = (
    transactions
    .dropna()
    .sort_values(by='sku_id')
)
    clear_transactions['client_id'] = clear_transactions['client_id'].astype(int)
    clear_transactions['hit'] = 1  # Клиент купил товар (нужно для разреженной матрицы)
    return clear_transactions.sample(size, random_state=seed)

In [39]:
transactions = pd.read_parquet(f'{path_to_data}transactions.parquet')
clear_transactions = clear_and_sample_transactions(transactions, 5 * 10**5)

rows, r_pos = np.unique(clear_transactions['client_id'].values, return_inverse=True)
cols, c_pos = np.unique(clear_transactions['sku_id'].values, return_inverse=True)

ui = csr_matrix((clear_transactions['hit'].values, (r_pos, c_pos)))
ui_T = ui.transpose(copy=True)

ui = normalize(ui, norm='l2', axis=1)
ui_T = normalize(ui_T, norm='l2', axis=1)

del rows, cols, r_pos, c_pos

In [40]:
actual = pd.DataFrame(
    clear_transactions
    .groupby('client_id')['sku_id']
    .apply(list)
    .values.tolist()
).values
del clear_transactions

In [41]:
sparse_predicted = ui @ ui_T @ ui
apk = []
for i, p in enumerate(tqdm(sparse_predicted, total=len(actual))):
    p = np.array(p.todense()).flatten()
    p = p[np.in1d(np.argsort(p), desired_sku)]
    a = actual[i][~np.isnan(actual[i])]
    apk.append(average_precision_at_k(a, p))

  0%|          | 0/198954 [00:00<?, ?it/s]

In [42]:
print(f'MAP@K: {np.mean(apk)}')

MAP@K: 2.392975224436398e-06


Как уже отмечалось ранее, по этой метрике нельзя судить, как хорошо модель решает задачу рекомендации товаров **определённых** групп. Поэтому не страшно, что она ниже, чем у бейзлайна.

# Выводы

Проведен разведочный анализ данных, предложены методы заполнения пропущенных значений.\
В качестве бейзлайна были взяты топ-20 самых популярных товаров из указанных в задании категорий.\
Опробованы два подхода к построению рекомендаций: векторное представление товаров и чека и применение случайных блужданий по матрице user-item.\
Отмечено, что метрика MAP@K не подходит для данной задачи, следовательно, делать выводы по её значениям нельзя, для всех трех подходов посчитана в информативных целях.


Поскольку последний подход предполагает наличие информации о прошлых покупках клиента, он не подойдёт для рекомендации "не представившимся" и новыми покупателям. В итоговой модели можно объединить два подхода: для неизвестных нам клиентов предсказывать топ-20 самых покупаемых (на практике это частый бейзлайн для холодных рекомендаций), а для известных использовать подход, представленный в пункте 6.

# Идеи

0. Качественнее заполнить пропуски в номенклатуре. Но, честно говоря, проще будет внутри **Организации** более качественно хранить данные о товарах
1. Векторные представления товаров и чеков можно строить умнее, например, с помощью encoder layer в нейронных сетях
2. Рекомендации можно улучшить с помощью [Market Basket Analysis](https://en.wikipedia.org/wiki/Affinity_analysis): для каждого товара найти товар(-ы), которые вероятнее всего купят вместе с ним (например, печенье к кофе)

У меня немного опыта работы с рекомендательными системами, историю транзакций использовал не на максимум. Возможно, хорошей идеей будет обучить классификатор на выборке с positive/negative sampling:

3. На всей истории из каждого чека можно удалить один случайный товар, сделать его таргетом (метка 1),
кроме этого, для каждого чека в таргет с меткой 0 (negative sampling) помещается товар, не принадлежащий этому чеку.\
Тогда можно будет обучить классификатор (данных много, они разнородные, табличные $\Rightarrow$ хорошо подойдёт градиентный бустинг, например, CatBoostClassifier), который по чеку и некоторому (лучше заранее отобранному) набору товаров будет предсказывать вероятности вхождения этих товаров в чек. \
То, как искать этот набор товаров заранее, - тоже отличный вопрос. В нашей задаче хотят увеличить продажи из конкретных групп, можно просто брать товары из этих групп